In [60]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import openpyxl

In [61]:
url = "https://www.imdb.com/chart/moviemeter/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

In [62]:
res = requests.get(url,headers=headers)
res

<Response [200]>

In [63]:
soup = BeautifulSoup(res.text,'html.parser')

In [64]:
# give the required html attribute name and the class name
tv_show = soup.find("ul",attrs={'class':'ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 iyTDQy compact-list-view ipc-metadata-list--base'})

In [65]:
tv_show

<ul class="ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 iyTDQy compact-list-view ipc-metadata-list--base" role="presentation"><li class="ipc-metadata-list-summary-item sc-4929eaf6-0 DLYcv cli-parent"><div class="sc-fe6afd55-0 dsShDt cli-poster-container"><div class="ipc-poster ipc-poster--base ipc-poster--media-radius ipc-poster--dynamic-width ipc-sub-grid-item ipc-sub-grid-item--span-2" role="group"><div aria-label="add to watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--s ipc-watchlist-ribbon--base ipc-watchlist-ribbon--loading ipc-watchlist-ribbon--onImage ipc-poster__watchlist-ribbon" data-testid="poster-watchlist-ribbon-add" role="button" tabindex="0"><svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg"><polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.2926049 24 31.7728343"></polygon><polygon clas

In [66]:
name =[]
year = []
duration = []
rank = []
rating = []
counts = []

In [67]:
for i in tv_show:
    #finding the movie name
    movie_name = i.find("div",attrs={"class":"ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-a69a4297-2 bqNXEn cli-title with-margin"}).a.h3.text
    name.append(movie_name)

    # finding the duration , released year , rank
    values = i.findAll("span",attrs={"class":"sc-300a8231-7 eaXxft cli-title-metadata-item"})

    movie_year = values[0].text
    year.append(movie_year)

    # Check if duration information exists before appending
    if len(values) > 1 and values[1].text.strip():  # Ensure there is a second value and it's not empty
        time = values[1].text.strip()
    else:
        time = "Not clear"
    duration.append(time)

    if len(values) > 2 and values[1].text.strip():  # Ensure there is a second value and it's not empty
        r = values[2].text.strip()
    else:
        r = "Not Ranked yet"
    rank.append(r)

    rate = i.find("span",attrs={"class":"ipc-rating-star--rating"})
    if rate and rate.text.strip():
        rate = rate.text.strip()
    else:
        rate = "Not Rated"
    rating.append(rate)

    rate_count = i.find("span",attrs={"class":"ipc-rating-star--voteCount"})
    if rate_count:
        rate_count = rate_count.get_text(strip=True).replace('\xa0', '').strip('()')
    else:
        rate_count = "NULL"
    counts.append(rate_count)


In [68]:
print(len(name),len(duration),len(rating),len(year),len(rank),len(counts))

25 25 25 25 25 25


In [79]:
df = pd.DataFrame({'Movie Name':name,'Released Year':year,'Duration':duration,'Rank':rank,'Rating':rating,'Rating Count':counts})

In [80]:
df = df.reset_index(drop=True)  # Reset index to start from 0 and drop the old index
df.index = range(1, len(df) + 1)  # Set the index to start from 1
df.index.name = 'S.No' 

In [81]:
df.head()

,Movie Name,Released Year,Duration,Rank,Rating,Rating Count
S.No,,,,,,
1,Carry-On,2024,1h 59m,PG-13,6.5,78K
2,Superman,2025,Not clear,Not Ranked yet,Not Rated,NULL
3,Red One,2024,2h 3m,PG-13,6.4,98K
4,Pushpa: The Rule - Part 2,2024,3h 21m,Not Ranked yet,6.5,35K
5,Kraven: The Hunter,2024,2h 7m,R,5.4,15K


In [83]:
df.to_csv('movie.csv')